# Part-1 Data Pre-Processing

In [220]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

from keras.models import load_model

In [221]:
model_File_Path = 'Infosys_RNN_Model.h5'

# Import Dataset

In [222]:
train_Dataset_File_Path = r'E.csv'
dataset_train = pd.read_csv(train_Dataset_File_Path)

training_set = dataset_train.iloc[:,1:2].values

# Feature Scaling
Normalisation

In [241]:
sc = MinMaxScaler(feature_range = (0,1))

training_set_Scaled = sc.fit_transform(training_set)

print (len(training_set_Scaled))

615


# Creating a data structure with 60 timesteps and 1 output

In [242]:
time_Step = 60

X_train = []
y_train = []

for i in range (time_Step, len(training_set_Scaled)):
    X_train.append(training_set_Scaled[i-time_Step: i, 0])
    y_train.append(training_set_Scaled[i,0])


X_train = np.array(X_train)
y_train = np.array(y_train)
    
# print (X_train)
# print (y_train)

# Reshaping


In [243]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Part-2 Building RNN

In [244]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import lstm, time

In [245]:
regressor = Sequential()

In [246]:
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1) ))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True, ))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True, ))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50 ))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

In [247]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Training 

In [248]:
regressor.fit(X_train, y_train, epochs = 10, batch_size = 32)

Epoch 1/10
555/555 [==============================] - 7s 12ms/step - loss: 0.1209
Epoch 2/10
555/555 [==============================] - 2s 3ms/step - loss: 0.0167
Epoch 3/10
555/555 [==============================] - 2s 4ms/step - loss: 0.0072
Epoch 4/10
555/555 [==============================] - 2s 3ms/step - loss: 0.0045
Epoch 5/10
555/555 [==============================] - 2s 3ms/step - loss: 0.0044
Epoch 6/10
555/555 [==============================] - 2s 3ms/step - loss: 0.0040
Epoch 7/10
555/555 [==============================] - 2s 3ms/step - loss: 0.0036
Epoch 8/10
555/555 [==============================] - 2s 3ms/step - loss: 0.0036
Epoch 9/10
555/555 [==============================] - 2s 3ms/step - loss: 0.0034
Epoch 10/10
555/555 [==============================] - 2s 3ms/step - loss: 0.0039


In [231]:
# regressor.save(model_File_Path)

In [232]:
# regressor = load_model(model_File_Path)

# Making Predicitions

In [249]:
test_Dataset_File_Path = r'F.csv'
dataset_test = pd.read_csv(test_Dataset_File_Path)
# print (type(dataset_test))
real_stock_price = dataset_test.iloc[:,1:2].values
print ((real_stock_price.shape))

(121, 1)


In [250]:
dataset_total = (dataset_train['Open Price'])
# print (type(dataset_total))
inputs = dataset_total[len(dataset_total) - time_Step : ].values
# print ((inputs.shape))
inputs = inputs.reshape(-1,1)
# print ((inputs.shape))
inputs = sc.transform(inputs)
# print ((inputs))


In [251]:
X_test = []

for i in range (time_Step, 110):
#     print (type(inputs[i-time_Step: i, 0]))
    X_test.append(inputs[i-time_Step: i, 0])

print(type(X_test))

X_test = np.array(X_test)
print((X_test.shape))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1) )

print((X_test.shape))


<class 'list'>
(50,)


IndexError: tuple index out of range

In [ ]:
predicted_Stock_Price = regressor.predict(X_test)
predicted_Stock_Price = sc.inverse_transform(predicted_Stock_Price)

# print ((predicted_Stock_Price))


In [252]:
# Predicting Stock price for next 100 days

future_Time_Steps = 100
data_last_60_days=[]
data_append_next_Prediction = 0

for i in range(len(dataset_test) - time_Step , len(dataset_test)):
    data_last_60_days.append((dataset_test.iloc[i,1]))
     

data_append_next_Prediction = data_last_60_days
data_last_60_days = np.array(data_last_60_days)
data_last_60_days = np.reshape(data_last_60_days, (time_Step, 1))

data_last_60_days = sc.transform(data_last_60_days)

print ((data_last_60_days.shape))


temp = []
temp.append(data_last_60_days[0:time_Step,0])

data_last_60_days = np.array(temp)
# print((data_last_60_days.shape))

data_last_60_days = np.reshape(data_last_60_days, (1, time_Step, 1) )
# print((data_last_60_days.shape))



predicted_Stock_Price = regressor.predict(data_last_60_days)
# predicted_Stock_Price = sc.inverse_transform(predicted_Stock_Price)

print(sc.inverse_transform(data_last_60_days))
print ((predicted_Stock_Price[0][0]))



for i in range (0,future_Time_Steps):
    data_append_next_Prediction.append(predicted_Stock_Price[0][0])
    
    data_last_60_days = []
    for i in range (len(data_append_next_Prediction) - time_Step, len(data_append_next_Prediction)):
        data_last_60_days.append(data_append_next_Prediction[i])
        
    data_append_next_Prediction = data_last_60_days
    data_last_60_days = np.array(data_last_60_days)
    data_last_60_days = np.reshape(data_last_60_days, (time_Step, 1))

    data_last_60_days = sc.transform(data_last_60_days)

#     print ((data_last_60_days.shape))


    temp = []
    temp.append(data_last_60_days[0:60,0])

    data_last_60_days = np.array(temp)
#     print((data_last_60_days.shape))

    data_last_60_days = np.reshape(data_last_60_days, (1, time_Step, 1) )
#     print((data_last_60_days.shape))



    predicted_Stock_Price = regressor.predict(data_last_60_days)
    predicted_Stock_Price = sc.inverse_transform(predicted_Stock_Price)

#     print ((predicted_Stock_Price[0][0]))


print ((data_append_next_Prediction) )


(60, 1)


ValueError: Found array with dim 3. Estimator expected <= 2.

# Visualising Result

In [ ]:
plt.plot(real_stock_price , color = 'red', label = 'Real Infosys Stock Price')
plt.plot(predicted_Stock_Price , color = 'blue', label = 'Predicted Infosys Stock Price')
plt.title('Stock Price')
plt.xlabel('Time In Days')
plt.ylabel('Stock Price')
plt.legend()
plt.show()